In [ ]:
import math

def ESTIMATE_AIRFOIL_PERFORMANCE(AIRFOIL_TYPE, ANGLE_OF_ATTACK_DEG, MAX_CAMBER_FRACTION=0):
    """
    Estimates the lift coefficient (CL) and moment coefficient about the quarter-chord (Cm_c/4)
    for an airfoil using Thin Airfoil Theory.

    This function supports symmetric airfoils and cambered airfoils (approximated by a parabolic camber line).

    Args:
        AIRFOIL_TYPE (str): Type of airfoil. Can be 'symmetric' or 'cambered'.
        ANGLE_OF_ATTACK_DEG (float): Angle of attack in degrees.
        MAX_CAMBER_FRACTION (float, optional): Maximum camber as a fraction of the chord.
                                               Only applicable for 'cambered' airfoils.
                                               For example, 0.02 for 2% camber. Defaults to 0.

    Returns:
        tuple: A tuple containing:
            - LIFT_COEFFICIENT (float): Lift Coefficient (CL)
            - MOMENT_COEFFICIENT_C4 (float): Moment Coefficient about the Quarter-Chord (Cm_c/4)
            - ZERO_LIFT_ANGLE_OF_ATTACK_DEG (float): Zero-Lift Angle of Attack in degrees
    """

    # Convert angle of attack from degrees to radians
    ANGLE_OF_ATTACK_RAD = math.radians(ANGLE_OF_ATTACK_DEG)

    # Initialize variables
    ZERO_LIFT_ANGLE_OF_ATTACK_RAD = 0.0  # Zero-Lift Angle of Attack in radians
    MOMENT_COEFFICIENT_C4 = 0.0         # Moment Coefficient about the Quarter-Chord

    # --- Calculations based on airfoil type ---
    if AIRFOIL_TYPE.lower() == 'symmetric':
        # For symmetric airfoils, the zero-lift angle of attack is 0,
        # and the moment coefficient about the quarter-chord is 0.
        ZERO_LIFT_ANGLE_OF_ATTACK_RAD = 0.0
        MOMENT_COEFFICIENT_C4 = 0.0
        print("\n--- Symmetric Airfoil Analysis ---")
    elif AIRFOIL_TYPE.lower() == 'cambered':
        # For a cambered airfoil approximated by a parabolic camber line
        # z_c(x) = 4 * h * x * (1 - x), where h = MAX_CAMBER_FRACTION.
        # The analytical results from Thin Airfoil Theory for this specific camber line are:
        # Zero-Lift Angle of Attack (alpha_L0) = -2 * h (in radians)
        # Moment Coefficient about the Quarter-Chord (Cm_c/4) = pi * h

        # If MAX_CAMBER_FRACTION is 0 or negative, the formulas will naturally result in 0 for alpha_L0 and Cm_c/4
        ZERO_LIFT_ANGLE_OF_ATTACK_RAD = -2 * MAX_CAMBER_FRACTION
        MOMENT_COEFFICIENT_C4 = math.pi * MAX_CAMBER_FRACTION
        print(f"\n--- Cambered Airfoil Analysis (Parabolic Camber, h={MAX_CAMBER_FRACTION:.3f}) ---")
    else:
        # This case should ideally not be reached if RUN_AIRFOIL_ESTIMATOR properly validates input
        print("Error: Invalid AIRFOIL_TYPE. This should not happen with proper user input validation.")
        return None, None, None # Still return None for safety, though it's less likely to be hit

    # Convert zero-lift angle of attack to degrees for output
    ZERO_LIFT_ANGLE_OF_ATTACK_DEG = math.degrees(ZERO_LIFT_ANGLE_OF_ATTACK_RAD)

    # --- General Thin Airfoil Theory Formulas ---
    # Lift Coefficient (CL)
    # The lift curve slope for thin airfoils is always 2*pi.
    # CL = 2 * pi * (ANGLE_OF_ATTACK_RAD - ZERO_LIFT_ANGLE_OF_ATTACK_RAD)
    LIFT_COEFFICIENT = 2 * math.pi * (ANGLE_OF_ATTACK_RAD - ZERO_LIFT_ANGLE_OF_ATTACK_RAD)

    return LIFT_COEFFICIENT, MOMENT_COEFFICIENT_C4, ZERO_LIFT_ANGLE_OF_ATTACK_DEG

# --- User Interaction ---
def RUN_AIRFOIL_ESTIMATOR():
    print("Welcome to the Airfoil Performance Estimator (Thin Airfoil Theory)")
    print("-----------------------------------------------------------------")

    while True:
        print("\nChoose an airfoil type:")
        print("1. Symmetric Airfoil")
        print("2. Cambered Airfoil (Generic Parabolic)")
        print("3. NACA 4-Digit Series (Approximate)")
        print("4. Exit")

        CHOICE = input("Enter your choice (1, 2, 3, or 4): ")

        if CHOICE == '1':
            AIRFOIL_TYPE = 'symmetric'
            MAX_CAMBER = 0 # Not applicable for symmetric
            print("\nYou selected Symmetric Airfoil.")
        elif CHOICE == '2':
            AIRFOIL_TYPE = 'cambered'
            while True:
                try:
                    MAX_CAMBER_INPUT = float(input("Enter maximum camber as a fraction of chord (e.g., 0.02 for 2%): "))
                    if MAX_CAMBER_INPUT >= 0:
                        MAX_CAMBER = MAX_CAMBER_INPUT
                        break
                    else:
                        print("Please enter a non-negative value for maximum camber.")
                except ValueError:
                    print("Invalid input. Please enter a number.")
            print(f"\nYou selected Generic Cambered Airfoil with {MAX_CAMBER*100:.1f}% camber.")
        elif CHOICE == '3':
            AIRFOIL_TYPE = 'cambered' # We will use the 'cambered' logic but derive MAX_CAMBER from NACA
            while True:
                NACA_INPUT = input("Enter NACA 4-digit series number (e.g., 2412): ")
                if len(NACA_INPUT) == 4 and NACA_INPUT.isdigit():
                    # Parse NACA 4-digit number: m p xx
                    # m = max camber (first digit) as % of chord
                    # p = position of max camber (second digit) as % of chord
                    # xx = max thickness (last two digits) as % of chord (not used in TAT for lift/moment)
                    M_PERCENT = int(NACA_INPUT[0])
                    # P_PERCENT = int(NACA_INPUT[1]) # Not directly used for simple parabolic approx

                    MAX_CAMBER = M_PERCENT / 100.0 # Convert % to fraction
                    print(f"\nYou selected NACA {NACA_INPUT} series.")
                    print(f"Approximating with a parabolic camber of {MAX_CAMBER*100:.1f}% for Thin Airfoil Theory.")
                    print("Note: Thin Airfoil Theory is a simplification and this approximation uses only the maximum camber.")
                    break
                else:
                    print("Invalid NACA 4-digit number. Please enter a 4-digit number (e.g., 2412).")
        elif CHOICE == '4':
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, or 4.")
            continue # Go back to the start of the loop

        while True:
            try:
                AOA_INPUT = float(input("Enter angle of attack in degrees: "))
                ANGLE_OF_ATTACK = AOA_INPUT
                break
            except ValueError:
                print("Invalid input. Please enter a number for angle of attack.")

        # Estimate performance using the chosen parameters
        LIFT_COEFFICIENT, MOMENT_COEFFICIENT_C4, ZERO_LIFT_ANGLE_OF_ATTACK_DEG = ESTIMATE_AIRFOIL_PERFORMANCE(
            AIRFOIL_TYPE=AIRFOIL_TYPE,
            ANGLE_OF_ATTACK_DEG=ANGLE_OF_ATTACK,
            MAX_CAMBER_FRACTION=MAX_CAMBER
        )

        # Removed the 'if LIFT_COEFFICIENT is not None' check as AIRFOIL_TYPE is always valid here.
        print("\n--- Estimated Performance ---")
        print(f"Angle of Attack: {ANGLE_OF_ATTACK} degrees")
        print(f"Zero-Lift Angle of Attack (ALPHA_L0): {ZERO_LIFT_ANGLE_OF_ATTACK_DEG:.2f} degrees")
        print(f"Estimated Lift Coefficient (CL): {LIFT_COEFFICIENT:.4f}")
        print(f"Estimated Moment Coefficient about Quarter-Chord (CM_C/4): {MOMENT_COEFFICIENT_C4:.4f}")

# Run the user interaction directly
RUN_AIRFOIL_ESTIMATOR()

Welcome to the Airfoil Performance Estimator (Thin Airfoil Theory)
-----------------------------------------------------------------

Choose an airfoil type:
1. Symmetric Airfoil
2. Cambered Airfoil (Generic Parabolic)
3. NACA 4-Digit Series (Approximate)
4. Exit

You selected Symmetric Airfoil.

--- Symmetric Airfoil Analysis ---

--- Estimated Performance ---
Angle of Attack: 10.0 degrees
Zero-Lift Angle of Attack (ALPHA_L0): 0.00 degrees
Estimated Lift Coefficient (CL): 1.0966
Estimated Moment Coefficient about Quarter-Chord (CM_C/4): 0.0000

Choose an airfoil type:
1. Symmetric Airfoil
2. Cambered Airfoil (Generic Parabolic)
3. NACA 4-Digit Series (Approximate)
4. Exit

You selected NACA 2251 series.
Approximating with a parabolic camber of 2.0% for Thin Airfoil Theory.
Note: Thin Airfoil Theory is a simplification and this approximation uses only the maximum camber.

--- Cambered Airfoil Analysis (Parabolic Camber, h=0.020) ---

--- Estimated Performance ---
Angle of Attack: 20.0 